In [31]:
import csv

def parse_redcap_codebook(csv_file):
    form_fields = {}

    form_name = None
    
    with open(csv_file, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            form_name = row['Form Name']
            fields = form_fields.get(form_name, [])
    
            field_name = row['Variable / Field Name']
            field_type = row['Field Type']
            field_label = row['Field Label']
            field_required = row['Required Field?']
            field_note = row['Field Note']
            field_choices = row['Choices, Calculations, OR Slider Labels'] # Choices for radio fields
            field_text_type = row['Text Validation Type OR Show Slider Number'] # For text field type, further detail on the exact type

            fields.append({
                'name': field_name,
                'type': field_type,
                'label': field_label,
                'required': field_required,
                'help_text': field_note,
                'choices': field_choices,
                'text_type': field_text_type
            })

            form_fields[form_name] = fields
    
    return form_fields


In [32]:
# text, radio, checkbox, dropdown and yesno will be mapped out to the default CharField, with radio, checkbox, dropdown and yesno having choices
text_type_mapping = {'integer': 'PositiveIntegerField',
                     'number': 'DecimalField',
                     'datetime_mdy': 'DateTimeField',
                     'date_mdy': 'DateField', }

field_type_mapping = {'notes': 'TextField',
                      'calc': 'DecimalField',
                      'file': 'FileField', }

In [33]:
def create_choices(str_choices):
    django_choices = []
    max_length = None
    choices_list = str_choices.split('|')
    for choices in choices_list:
        choice = [str(value).strip() for value in choices.split(',', 1)]
        if not max_length or (max_length and len(choice[0]) > max_length):
            max_length = len(choice[0])

        django_choices.append(tuple(choice))
    return tuple(django_choices), max_length

In [34]:
import json

def generate_django_models(fields, form_name):
    models_code = ""
    for field in fields:
        field_name = field['name']
        field_type = field['type']
        field_label = field['label'].replace('\n', '')
        text_type = field['text_type']
        field_required = field['required']
        field_help = field['help_text'].replace('\n', '')
        
        # Determine Django field type based on field type from REDCap
        django_field_type = field_type_mapping.get(field_type, 'CharField')  # Default to CharField
        
        if field_type == 'text' and text_type:
            django_field_type = text_type_mapping.get(text_type, django_field_type)
        
        # Generate Django model field code
        field_open = f"  {field_name} = models.{django_field_type}(\n"

        field_verbose = f"  verbose_name={json.dumps(field_label)},\n".replace("'", "\'")
        required_options = ''
        extra_options = ''

        # define CharField attrs
        if django_field_type == 'CharField':
            choices = None
            field_length = 100
            if field_type == 'yesno':
                choices, field_length = create_choices('1, Yes | 0, No')
            if field_type in ['radio', 'checkbox', 'dropdown', ]:
                choices, field_length = create_choices(field['choices'])

            # if choices and choices not in existing_choices:
            #     choices = choices
            char_options = f"  max_length={field_length},\n"
            extra_options = char_options + f"  choices={choices},\n" if choices else char_options

        # define DecimalField attrs
        if django_field_type == 'DecimalField':
            extra_options = 'decimal_places=2, max_digits=8,\n'
        
        # define required options
        if field_required != 'y':
            required_options = '  blank=True, null=True,\n'

        field_close = f"  help_text={json.dumps(field_help)}, )"
        def_model = field_open + field_verbose + extra_options + required_options + field_close
        models_code += def_model + "\n\n"
    
    django_models_code = f"class {form_name.capitalize()}(models.Model):\n\n{models_code}"
    return django_models_code


In [35]:
def write_to_django_models_file(models_code, filename):
    with open(filename, 'w') as f:
        f.write(models_code)

In [36]:
file_path = '/Users/tonyonkgopotserichard/Downloads/tsepamo_2.csv'
form_fields = parse_redcap_codebook(file_path)
for form_name, fields in form_fields.items():
    models_code = generate_django_models(fields, form_name)
    write_to_django_models_file(models_code, f'../models/{form_name}.py')